In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -Uq "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.6 MB/s eta 0:00:0000:01


In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient

client = genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY"))

In [5]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

Evaluation

Start by downloading the PDF to the notebook environment, and uploading that copy for use with the Gemini API.

In [6]:
!wget -nv -O gemini.pdf https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf

document_file = client.files.upload(file='gemini.pdf')

2025-05-20 10:43:15 URL:https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf [7228817/7228817] -> "gemini.pdf" [1]


Summarise a document

In [7]:
request = 'Tell me about the training process used here.'

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.0)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Based on the document you provided, here's a breakdown of the training process used for Gemini 1.5 Pro:

**1. Model Architecture:**

*   Gemini 1.5 Pro is a **sparse mixture-of-experts (MoE) Transformer-based model.** This means it builds upon the Transformer architecture (Vaswani et al., 2017) but incorporates a MoE structure.
*   MoE models use a **learned routing function** to direct inputs to a subset of the model's parameters for processing. This allows the model to have a large total parameter count while only activating a portion of those parameters for any given input.

**2. Training Data:**

*   The model is trained on a **variety of multimodal and multilingual data.**
*   The pre-training dataset includes data sourced from many different domains, including **web documents, code, images, audio, and video content.**

**3. Training Infrastructure:**

*   Gemini 1.5 Pro is trained on **multiple 4096-chip pods of Google's TPUv4 accelerators**, distributed across multiple datacenters.

**4. Training Process:**

*   **Pre-training:** The model is initially pre-trained on the large, diverse dataset mentioned above.
*   **Instruction Tuning:** After pre-training, Gemini 1.5 Pro is fine-tuned on a collection of multimodal data containing paired instructions and appropriate responses.
*   **Human Preference Tuning:** Further tuning is performed based on human preference data.

**Key Aspects and Innovations:**

*   **Long Context Understanding:** A series of significant architecture changes enable long-context understanding of inputs up to 10 million tokens without degrading performance.
*   **Efficiency:** Improvements across the entire model stack (architecture, data, optimization, and systems) allow Gemini 1.5 Pro to achieve comparable quality to Gemini 1.0 Ultra while using significantly less training compute and being significantly more efficient to serve.
*   **Multimodality:** The model is natively multimodal and supports interleaving of data from different modalities (audio, visual, text, code) in the same input sequence.

In summary, the training process involves a combination of large-scale pre-training on diverse multimodal data, followed by instruction tuning and human preference tuning, all leveraging a MoE architecture and Google's TPU infrastructure. A key focus is on enabling the model to handle extremely long contexts effectively.


Define an evaluator

In [9]:
import enum

# Define the evaluation prompt
SUMMARY_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user input and an AI-generated responses.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step-by-step explanations for your rating, and only choose ratings from the Rating Rubric.

# Evaluation
## Metric Definition
You will be assessing summarization quality, which measures the overall ability to summarize text. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a summarization task and the context to be summarized are provided in the user prompt. The response should be shorter than the text in the context. The response should not contain information that is not present in the context.

## Criteria
Instruction following: The response demonstrates a clear understanding of the summarization task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context. The response does not reference any outside information.
Conciseness: The response summarizes the relevant details in the original text without a significant loss in key information without being too verbose or terse.
Fluency: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The summary follows instructions, is grounded, is concise, and fluent.
4: (Good). The summary follows instructions, is grounded, concise, and fluent.
3: (Ok). The summary mostly follows instructions, is grounded, but is not very concise and is not fluent.
2: (Bad). The summary is grounded, but does not follow the instructions.
1: (Very bad). The summary is not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs

### Prompt
{prompt}

## AI-generated Response
{response}
"""

# Define a structured enum class to capture the result.
class SummaryRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


def eval_summary(prompt, ai_response):
  """Evaluate the generated summary against the prompt used."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=SummaryRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_summary(prompt=[request, document_file], ai_response=summary)
Markdown(text_eval)

Rating: 5

Explanation:
The response gives a great summary of the training process of Gemini 1.5 Pro. The response demonstrates a clear understanding of the summarization task instructions, satisfying all of the instruction's requirements. The response contains information included only in the context. The response summarizes the relevant details in the original text without a significant loss in key information without being too verbose or terse. The response is well-organized and easy to read.

In [10]:
struct_eval

<SummaryRating.VERY_GOOD: '5'>

In [13]:
new_prompt = "Explain like I'm 5 the training process"
# Try:
#  ELI5 the training process
#  Summarise the needle/haystack evaluation technique in 1 line
#  Describe the model architecture to someone with a civil engineering degree
#  What is the best LLM?

if not new_prompt:
  raise ValueError("Try setting a new summarisation prompt.")


def run_and_eval_summary(prompt):
  """Generate and evaluate the summary using the new prompt."""
  summary = summarise_doc(new_prompt)
  display(Markdown(summary + '\n-----'))

  text, struct = eval_summary([new_prompt, document_file], summary)
  display(Markdown(text + '\n-----'))
  print(struct)

run_and_eval_summary(new_prompt)

Okay, I can explain the training process of a large language model like Gemini 1.5 Pro in a way that a 5-year-old can understand.

Imagine you have a puppy, and you want to teach it to understand and do tricks.

1.  **Lots and Lots of Examples:** First, you show the puppy tons and tons of things. You show it pictures of cats, dogs, cars, and houses. You also read it lots of stories and tell it lots of things. This is like giving the computer model a huge pile of books, pictures, videos, and sounds.

2.  **Learning the Rules:** The puppy starts to notice patterns. It sees that cats have pointy ears and dogs have floppy ears. It learns that when you say "sit," it should put its bottom on the ground. The computer model does the same thing. It looks for patterns in all the information it has and learns the rules of language and the world.

3.  **Practicing and Getting Feedback:** Now, you ask the puppy to do things. You say "sit," and if it sits, you give it a treat and say "Good dog!" If it doesn't sit, you gently show it what to do. The computer model also practices. It tries to answer questions, write stories, and translate languages. If it does a good job, it gets a "treat" (a small adjustment to its rules). If it makes a mistake, it gets corrected.

4.  **Getting Better and Better:** The puppy keeps practicing and getting feedback, and it gets better and better at understanding and doing tricks. The computer model does the same thing. It keeps learning from its mistakes and getting better at all the different tasks it can do.

So, basically, training a large language model is like teaching a puppy, but instead of treats and belly rubs, you use math and computers to help it learn! The more examples you give it, and the more feedback it gets, the smarter it becomes.
-----

## Evaluation
STEP 1: The response follows the instructions of explaining the training process like I'm 5. The response provides a good analogy for a child to follow. The response is grounded to the document provided. The response is concise and fluent.

STEP 2: The response is very good and receives a score of 5.

## Rating:
5

-----

SummaryRating.VERY_GOOD


Pointwise evaluation

The technique used above, where you evaluate a single input/output pair against some criteria is known as pointwise evaluation. This is useful for evaluating singular outputs in an absolute sense, such as "was it good or bad?"

In [14]:
import functools

# Try these instructions, or edit and add your own.
terse_guidance = "Answer the following question in a single sentence, or as close to that as possible."
moderate_guidance = "Provide a brief answer to the following question, use a citation if necessary, but only enough to answer the question."
cited_guidance = "Provide a thorough, detailed answer to the following question, citing the document and supplying additional background information as much as possible."
guidance_options = {
    'Terse': terse_guidance,
    'Moderate': moderate_guidance,
    'Cited': cited_guidance,
}

questions = [
    # Un-comment one or more questions to try here, or add your own.
    # Evaluating more questions will take more time, but produces results
    # with higher confidence. In a production system, you may have hundreds
    # of questions to evaluate a complex system.

    # "What metric(s) are used to evaluate long context performance?",
    "How does the model perform on code tasks?",
    "How many layers does it have?",
    # "Why is it called Gemini?",
]

if not questions:
  raise NotImplementedError('Add some questions to evaluate!')


@functools.cache
def answer_question(question: str, guidance: str = '') -> str:
  """Generate an answer to the question using the uploaded document and guidance."""
  config = types.GenerateContentConfig(
      temperature=0.0,
      system_instruction=guidance,
  )
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[question, document_file],
  )

  return response.text


answer = answer_question(questions[0], terse_guidance)
Markdown(answer)

Gemini 1.5 Pro performs well on code tasks, surpassing Gemini 1.0 Ultra on Natural2Code and showing improvements in coding capabilities compared to previous Gemini models.


In [15]:
import enum

QA_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user prompt and an AI-generated responses.
You should first read the user prompt carefully for analyzing the task, and then evaluate the quality of the responses based on and rules provided in the Evaluation section below.

# Evaluation
## Metric Definition
You will be assessing question answering quality, which measures the overall quality of the answer to the question in the user prompt. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a question-answering task is provided in the user prompt. The response should not contain information that is not present in the context (if it is provided).

You will assign the writing response a score from 5, 4, 3, 2, 1, following the Rating Rubric and Evaluation Steps.
Give step-by-step explanations for your scoring, and only choose scores from 5, 4, 3, 2, 1.

## Criteria Definition
Instruction following: The response demonstrates a clear understanding of the question answering task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context if the context is present in the user prompt. The response does not reference any outside information.
Completeness: The response completely answers the question with sufficient detail.
Fluent: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The answer follows instructions, is grounded, complete, and fluent.
4: (Good). The answer follows instructions, is grounded, complete, but is not very fluent.
3: (Ok). The answer mostly follows instructions, is grounded, answers the question partially and is not very fluent.
2: (Bad). The answer does not follow the instructions very well, is incomplete or not fully grounded.
1: (Very bad). The answer does not follow the instructions, is wrong and not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness,completeness, and fluency according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs
### Prompt
{prompt}

## AI-generated Response
{response}
"""

class AnswerRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


@functools.cache
def eval_answer(prompt, ai_response, n=1):
  """Evaluate the generated answer against the prompt/question used."""
  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=QA_PROMPT.format(prompt=[prompt, document_file], response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=AnswerRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_answer(prompt=questions[0], ai_response=answer)
display(Markdown(text_eval))
print(struct_eval)

STEP 1: Assess the response in aspects of instruction following, groundedness, completeness, and fluency according to the criteria.

Instruction Following: The answer fulfills the prompt's instructions.
Groundedness: The information provided in the answer is based on the document that was provided.
Completeness: The response provides an adequate answer to the prompt.
Fluency: The response is well-written and easy to understand.

STEP 2: Score based on the rubric.

The response is very good because it follows instructions, is grounded, complete, and fluent.

Final Answer: (5)


AnswerRating.VERY_GOOD


In [17]:
import collections
import itertools

# Number of times to repeat each task in order to reduce error and calculate an average.
# Increasing it will take longer but give better results, try 2 or 3 to start.
NUM_ITERATIONS = 1

scores = collections.defaultdict(int)
responses = collections.defaultdict(list)

for question in questions:
  display(Markdown(f'## {question}'))
  for guidance, guide_prompt in guidance_options.items():

    for n in range(NUM_ITERATIONS):
      # Generate a response.
      answer = answer_question(question, guide_prompt)

      # Evaluate the response (note that the guidance prompt is not passed).
      written_eval, struct_eval = eval_answer(question, answer, n)
      print(f'{guidance}: {struct_eval}')

      # Save the numeric score.
      scores[guidance] += int(struct_eval.value)

      # Save the responses, in case you wish to inspect them.
      responses[(guidance, question)].append((answer, written_eval))

## How does the model perform on code tasks?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD


## How many layers does it have?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.OK
Cited: AnswerRating.VERY_GOOD


In [18]:
for guidance, score in scores.items():
  avg_score = score / (NUM_ITERATIONS * len(questions))
  nearest = AnswerRating(str(round(avg_score)))
  print(f'{guidance}: {avg_score:.2f} - {nearest.name}')

Terse: 5.00 - VERY_GOOD
Moderate: 4.00 - GOOD
Cited: 5.00 - VERY_GOOD


Pairwise evaluation

The pointwise evaluation prompt used in the previous step has 5 levels of grading in the output. This may be too coarse for your system, or perhaps you wish to improve on a prompt that is already "very good".

Another approach to evaluation is to compare two outputs against each other. This is pairwise evaluation, and is a key step in ranking and sorting algorithms, which allows you to use it to rank your prompts either instead of, or in addition to the pointwise approach.

In [19]:
QA_PAIRWISE_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by two AI models. We will provide you with the user input and a pair of AI-generated responses (Response A and Response B). You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.

You will first judge responses individually, following the Rating Rubric and Evaluation Steps. Then you will give step-by-step explanations for your judgment, compare results to declare the winner based on the Rating Rubric and Evaluation Steps.

# Evaluation
## Metric Definition
You will be assessing question answering quality, which measures the overall quality of the answer to the question in the user prompt. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a question-answering task is provided in the user prompt. The response should not contain information that is not present in the context (if it is provided).

## Criteria
Instruction following: The response demonstrates a clear understanding of the question answering task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context if the context is present in the user prompt. The response does not reference any outside information.
Completeness: The response completely answers the question with sufficient detail.
Fluent: The response is well-organized and easy to read.

## Rating Rubric
"A": Response A answers the given question as per the criteria better than response B.
"SAME": Response A and B answers the given question equally well as per the criteria.
"B": Response B answers the given question as per the criteria better than response A.

## Evaluation Steps
STEP 1: Analyze Response A based on the question answering quality criteria: Determine how well Response A fulfills the user requirements, is grounded in the context, is complete and fluent, and provides assessment according to the criterion.
STEP 2: Analyze Response B based on the question answering quality criteria: Determine how well Response B fulfills the user requirements, is grounded in the context, is complete and fluent, and provides assessment according to the criterion.
STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
STEP 5: Output your assessment reasoning in the explanation field.

# User Inputs and AI-generated Responses
## User Inputs
### Prompt
{prompt}

# AI-generated Response

### Response A
{baseline_model_response}

### Response B
{response}
"""


class AnswerComparison(enum.Enum):
  A = 'A'
  SAME = 'SAME'
  B = 'B'


@functools.cache
def eval_pairwise(prompt, response_a, response_b, n=1):
  """Determine the better of two answers to the same prompt."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=QA_PAIRWISE_PROMPT.format(
          prompt=[prompt, document_file],
          baseline_model_response=response_a,
          response=response_b)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=AnswerComparison,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


question = questions[0]
answer_a = answer_question(question, terse_guidance)
answer_b = answer_question(question, cited_guidance)

text_eval, struct_eval = eval_pairwise(
    prompt=question,
    response_a=answer_a,
    response_b=answer_b,
)

display(Markdown(text_eval))
print(struct_eval)

## Judge Feedback
STEP 1: Analyze Response A based on the question answering quality criteria:
Response A is a very brief and succinct answer. It only highlights one or two points from the document.

STEP 2: Analyze Response B based on the question answering quality criteria:
Response B takes a much more structured approach and highlights several important points regarding the model's performance on code tasks.

STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
Response B is the much better answer here. It highlights many more relevant details than Response A does.

STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
B

STEP 5: Output your assessment reasoning in the explanation field.
Response B is more thorough and explains the performance of the model in greater detail, whereas Response A simply skims the surface.

AnswerComparison.B
